In [1]:
import pandas as pd
import sqlite3


In [2]:
def preprocess_data(df):
    # Remove duplicate messages
    df = df.drop_duplicates()

    # Normalize text: lowercase
    df["text"] = df["text"].str.lower()

    # Remove special characters (optional)
    df["text"] = df["text"].str.replace(r"[^\w\s]", "", regex=True)

    df

    # Structure data for chatbot (input-response pairs)
    conversations = []
    for i in range(len(df) - 1):
        if df.iloc[i]["sender"] == "You" and df.iloc[i + 1]["sender"] == "Me":
            conversations.append({"input": df.iloc[i]["text"], "response": df.iloc[i + 1]["text"]})

    # Convert to DataFrame
    chatbot_data = pd.DataFrame(conversations)

    # Save processed data for chatbot training
    chatbot_data.to_csv("../data/chatbot_data.csv", index=False)

    print("Preprocessing complete! Saved as chatbot_data.csv")

In [3]:
with sqlite3.connect("../data/texts.db") as conn:
    query = '''
    SELECT m.text, m.date, h.id as sender
    FROM message m
    JOIN handle h ON m.handle_id = h.ROWID
    '''
    # df_texts = pd.read_sql_query(query, conn)
    # df_texts.to_csv('texts.csv')  # Export to CSV

    query_myself = '''
    SELECT 
    CASE 
        WHEN "is_from_me" = 0 THEN 'You'
        WHEN "is_from_me" = 1 THEN 'Me'
    END as sender,
    "text"
    FROM "main"."message"
    WHERE length(text) > 1
    AND ROWID > 100000
    AND handle_id IS NOT 0
    AND handle_id IN (
        SELECT handle_id
        FROM "main"."message"
        GROUP BY handle_id
        HAVING COUNT(*) >= 100
    )
    ORDER BY "handle_id", "ROWID" ASC
    LIMIT 0, 499999;
    '''

    df_my_texts = pd.read_sql_query(query_myself, conn)
    df_my_texts
    preprocess_data(df_my_texts)

/var/folders/gg/7n0jp1r94z564swcxk0rk4w40000gn/T/ipykernel_18158/3597340635.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].str.lower()
/var/folders/gg/7n0jp1r94z564swcxk0rk4w40000gn/T/ipykernel_18158/3597340635.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].str.replace(r"[^\w\s]", "", regex=True)


Preprocessing complete! Saved as chatbot_data.csv


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

df = pd.read_csv("../data/chatbot_data.csv")

def format_dialog(df):
    conversations = []
    for _, row in df.iterrows():
        conversations.append(f"User: {row['input']}\nBot: {row['response']}")
    return conversations

formatted_texts = format_dialog(df)

/Users/charley/Programming/personal/charbot/venv_charbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sklearn.model_selection import train_test_split

# Split into train and eval datasets
train_texts, eval_texts = train_test_split(formatted_texts, test_size=0.1)  # 10% for evaluation

In [ ]:
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Tokenize with padding and truncation
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Shift the input_ids to create labels for the model
train_labels = train_encodings["input_ids"].clone()  # Clone input_ids for labels
eval_labels = eval_encodings["input_ids"].clone()

# Shift the labels to the right (for causal language modeling)
train_labels[:, :-1] = train_encodings["input_ids"][:, 1:].clone()
eval_labels[:, :-1] = eval_encodings["input_ids"][:, 1:].clone()

# Replace the last token in labels with -100 to ignore padding tokens
train_labels[train_labels == tokenizer.pad_token_id] = -100
eval_labels[eval_labels == tokenizer.pad_token_id] = -100

# Add the labels to the encodings
train_encodings["labels"] = train_labels
eval_encodings["labels"] = eval_labels
# Convert into Dataset objects
train_dataset = Dataset.from_dict({key: val.tolist() for key, val in train_encodings.items()})
eval_dataset = Dataset.from_dict({key: val.tolist() for key, val in eval_encodings.items()})

class ChatDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings.input_ids)
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

# train_dataset = ChatDataset(train_dataset)

training_args = TrainingArguments(
    output_dir="./models/charbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="epoch"
)


In [6]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Tokenize with padding and truncation
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Shift the input_ids to create labels for the model
train_labels = train_encodings["input_ids"].clone()  # Clone input_ids for labels
eval_labels = eval_encodings["input_ids"].clone()

# Shift the labels to the right (for causal language modeling)
train_labels[:, :-1] = train_encodings["input_ids"][:, 1:].clone()
eval_labels[:, :-1] = eval_encodings["input_ids"][:, 1:].clone()

# Replace the last token in labels with -100 to ignore padding tokens
train_labels[train_labels == tokenizer.pad_token_id] = -100
eval_labels[eval_labels == tokenizer.pad_token_id] = -100

# Add the labels to the encodings
train_encodings["labels"] = train_labels
eval_encodings["labels"] = eval_labels
# Convert into Dataset objects
train_dataset = Dataset.from_dict({key: val.tolist() for key, val in train_encodings.items()})
eval_dataset = Dataset.from_dict({key: val.tolist() for key, val in eval_encodings.items()})

class ChatDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings.input_ids)
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

# train_dataset = ChatDataset(train_dataset)

training_args = TrainingArguments(
    output_dir="charbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="epoch"
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.64s/it]
/Users/charley/Programming/personal/charbot/venv_charbot/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()
model.save_pretrained("models/charbot_model_medium")
tokenizer.save_pretrained("models/charbot_model_medium")

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset
# )

# trainer.train()
# model.save_pretrained("models/charbot_model_small")
# tokenizer.save_pretrained("models/charbot_model_small")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,4.925000,5.146930
2,4.526400,5.109916
3,4.337700,5.146240


('models/charbot_model_small/tokenizer_config.json',
 'models/charbot_model_small/special_tokens_map.json',
 'models/charbot_model_small/vocab.json',
 'models/charbot_model_small/merges.txt',
 'models/charbot_model_small/added_tokens.json',
 'models/charbot_model_small/tokenizer.json')

In [24]:
# Load trained chatbot
model = AutoModelForCausalLM.from_pretrained("models/charbot_model_small")
tokenizer = AutoTokenizer.from_pretrained("models/charbot_model_small")

# Function to chat with your model
def chat_with_bot():
    print("Charbot is ready! Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        input_text = f"User: {user_input}\nBot:"
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        response_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
        print(f"Bot: {response}")

chat_with_bot()

Charbot is ready! Type 'exit' to stop.
Bot:  you so at r r r r r goodmg so to  you so  you so at r r goodmgmg so to you so  you so at r r goodmgmg so to you so Bot i you so at r r goodmgmg so to you so  you so at r r goodmgmgmg so to you so Bot i you so at r r
Bot:  loveuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu
Bot:  you so at r r r r r goodmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmgmg
Bot: hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
